# Take a bunch of pictures from the drone on-board camera and assemble a map from them

In [4]:
FAMES_FOLDER = 'mapping_frames'

In [ ]:
import os


def list_directory_contents():
    items = []
    try:
        contents = os.listdir(f"{FAMES_FOLDER}/")
        for item in contents:
            items.append(f"{FAMES_FOLDER}/{item}")
        return items
    except Exception as e:
        print(f"An error occurred: {e}")


frames = list_directory_contents()

print("Contents of the FAMES folder:", frames)

Contents of the FAMES folder: ['mapping_frames/frame_20250710_203508.jpg', 'mapping_frames/frame_20250710_203509.jpg', 'mapping_frames/frame_20250710_203452.jpg', 'mapping_frames/frame_20250710_203503.jpg', 'mapping_frames/frame_20250710_203517.jpg', 'mapping_frames/frame_20250710_203515.jpg', 'mapping_frames/frame_20250710_203514.jpg', 'mapping_frames/frame_20250710_203500.jpg', 'mapping_frames/frame_20250710_203510.jpg', 'mapping_frames/frame_20250710_203511.jpg', 'mapping_frames/frame_20250710_203505.jpg', 'mapping_frames/frame_20250710_203512.jpg']


In [14]:
import cv2

class FramesMapper:
    def __init__(self, frames):
        self.frames = []
        self.keypoints = []
        self.descriptors = []
        self.images = []
        
        self.orb = cv2.ORB_create(nfeatures=1000)
        self.matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        
    def load_frames(self, frames):
        # Load frames from the provided list
        for frame in frames:
            if os.path.exists(frame):
                img = cv2.imread(frame, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    self.frames.append(frame)
                    self.images.append(img)
                    keypoints, descriptors = self.orb.detectAndCompute(img, None)
                    self.keypoints.append(keypoints)
                    self.descriptors.append(descriptors)
            else:
                print(f"Frame {frame} does not exist.")
        print("Loaded frames:", len(self.frames))
        
        # Extract ORB keypoints and descriptors
        for image in self.images:
            keypoints, descriptors = self.orb.detectAndCompute(image, None)
            self.keypoints.append(keypoints)
            self.descriptors.append(descriptors)
        print(f"Extracted keypoints: {len(self.keypoints)}")
    
    def match_features(self, desc1, desc2):
        """Match features between two frames."""
        if desc1 is None or desc2 is None:
            return []
            
        matches = self.matcher.match(desc1, desc2)
        matches = sorted(matches, key=lambda x: x.distance)
        
        good = []
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good.append(m)
        return good
            

